In [1]:
from datasets import load_dataset,Dataset
from datasets.features import Features,Value
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import seaborn as sns
import matplotlib.pylab as plt
import os
from datasets.utils import disable_progress_bar
from bertopic import BERTopic
import numpy as np
from IPython.display import Image
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from datasets.load import load_from_disk

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
df = load_dataset("rjac/all-the-news-2-1-Component-one")
df = df.filter(lambda batch: batch["article"] is not None)

In [ ]:
def paragraph_tokenizer(batch):
    articles = batch["article"]
    article_idx = batch["article_idx"]
    years = batch["year"]
    months = batch["month"]

    sentence_len = 10

    paragraph_list = []
    paragraph_idx = []
    paragraph_year = []
    paragraph_month = []
    paragraph_position = []

    for idx,year,month, text in zip(article_idx,years,months,articles):
        
        sentences = sent_tokenize(text)
        number_of_paragraphs = int(len(sentences)/sentence_len)

        paragraphs = []

        for i in range(number_of_paragraphs+1):
            start = i*sentence_len
            end = start+sentence_len

            paragraph_list.append(' '.join(sentences[start:end]))
            paragraph_idx.append(idx)
            paragraph_year.append(year)
            paragraph_month.append(month)
            paragraph_position.append(i)
            
    return {"article_idx":paragraph_idx,"year":paragraph_year,"month":paragraph_month,"paragraph_position":paragraph_position,"paragraph":paragraph_list}

In [ ]:
paragraph_df = df.map(paragraph_tokenizer,batched=True,num_proc=os.cpu_count(),remove_columns=df["train"].column_names)

In [ ]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
paragraph_df = paragraph_df.map(lambda batch: {"embedding":sentence_model.encode(batch["paragraph"])},batched=True,num_proc=1)

In [ ]:
paragraph_df = paragraph_df.filter(lambda batch: [len(text)>10 for text in batch["paragraph"]],batched=True,num_proc=os.cpu_count())

In [ ]:
paragraph_df.push_to_hub("rjac/all-the-news-2-1-Component-one-sentence-embedding",max_shard_size="1GB",private=False,token=os.getenv("AUTH_TOKEN"))